In [1]:
!pip install xmltodict requests_toolbelt owslib

In [ ]:
import requests
import os
import urllib
import json
import re
import string
import xmltodict
from requests_toolbelt.multipart.decoder import MultipartDecoder
from tqdm.auto import tqdm

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
connectId = "228b427a-fded-4f00-9602-1225879e86a6"

In [ ]:
def getImage(payload):
    baseurl = "https://securewatch.maxar.com/mapservice/wmsaccess?"
    url = baseurl+urllib.parse.urlencode(payload)
    headers = {}
    r = requests.request("GET", url, headers=headers, data=payload)
    return(r.content)

def getBbox(coord_dict):
    bbox_list = list()
    for feature in coord_dict["features"]:
        coord_list = feature["geometry"]["coordinates"][0][0]
        uid = feature["properties"]["uid"]
        lat = list()
        lon = list()
        for coord in coord_list:
            lon.append(coord[0])
            lat.append(coord[1])

        maxlat = max(lat)
        minlat = min(lat)
        maxlon = max(lon)
        minlon = min(lon)
        bbox_list.append([[minlat, minlon],[maxlat,maxlon],uid])
        
    return bbox_list

def getImageWCS(payload):
    baseurl = "https://securewatch.maxar.com/deliveryservice/wcsaccess?"
    url = baseurl+urllib.parse.urlencode(payload)
    headers = {}
    r = requests.request("GET", url, headers=headers, data=payload)
    return(r)

def getFeaturesWFS(payload):
    base_url = 'https://securewatch.maxar.com/catalogservice/wfsaccess?'
    url = base_url+urllib.parse.urlencode(payload)
    headers = {}
    r = requests.request("GET", url, headers=headers, data=payload)
    return(r.content)

def selectBestImage(short_list):
    id = short_list[0]["id"]
    return id

Import .Json File

# Get list AOI
 - read in json file
 - parse file to get list of bounding boxes
 - convert to [minlat, minlon, maxlat, maxlon] format

In [ ]:
file = open('./5_sites.geojson','r')
json_file = file.read()
coord_dict = json.loads(json_file)

In [ ]:
coord_dict["features"][0]["properties"]["uid"]

In [ ]:
bboxes = getBbox(coord_dict)
bboxes

# Get images using WFS and WCS
- get image id's using WFS Service
- parse and simplify WFS response
- select image based on priorities
- return get image id for selected image
- pull image using WCS
- save in file with uid as name

In [ ]:
bboxes

In [ ]:
for bbox in tqdm(bboxes):
    #get list of images from WFS Service
    payload={
    "connectId": connectId,
    "Service":"WFS",
    "Request":"GetFeature",
    "version":"1.1.0",
    "srsName":"EPSG:4326"
    }
    
    bbox_coords = str(bbox[0][0]) + "," + str(bbox[0][1]) + "," + str(bbox[1][0]) + "," + str(bbox[1][1])

    # apparently the order matters of parameters
    payload["BBOX"]= bbox_coords
    payload["typeName"] = "FinishedFeature"
    payload["WIDTH"] = "1102"
    payload["HEIGHT"] = "712"
    FeatureCollectionXML = getFeaturesWFS(payload)
    FeatureCollection = xmltodict.parse(FeatureCollectionXML)
    
    #Translate resulting list into simplified list
    short_list = list()
    for item in FeatureCollection["wfs:FeatureCollection"]['gml:featureMembers']['DigitalGlobe:FinishedFeature']:
        if item['DigitalGlobe:productType'] == "Pan Sharpened Natural Color":
            image_details = dict()b
            image_details["id"] = item['@gml:id']
            image_details["date"] = item["DigitalGlobe:acquisitionDate"]
            image_details["type"] = item["DigitalGlobe:productType"]
            image_details["cloud_covers"] = item["DigitalGlobe:cloudCover"]
            image_details["offNadir"] = item["DigitalGlobe:offNadirAngle"]
            image_details["gsd"] = item["DigitalGlobe:groundSampleDistance"]
            short_list.append(image_details)
    logging.info("Short List: " + str(short_list))
    
    #if there are images in the list
    if len(short_list) > 0:
        #select best image
        image_id = selectBestImage(short_list)
        logging.info("Selected Image ID:" + str(image_id))

        #go get this images from WCS Service
        WCSpayload={
            "connectId":connectId,
            "Service":"WCS",
            "Request":"GetCoverage",
            "version":"1.1.0",
            "srsName":"EPSG:4326",
            "typeName":"FinishedFeature",
            "WIDTH":"1102",
            "HEIGHT":"712",
            "format":"image/geotiff"}

        WCSpayload["Identifier"] = image_id
        WCSpayload["boundingBox"] = bbox_coords
        
        response = getImageWCS(WCSpayload)
        decoder = MultipartDecoder.from_response(response)
        parts = decoder.parts
        basename = str(bbox[2])

        with open(basename + '.xml', 'wb') as f:
            f.write(parts[0].content)
        with open(basename + '.tiff', 'wb') as f:
            f.write(parts[1].content)
        
